In [14]:
import os
import shutil
from roboflow import Roboflow
from PIL import Image
import cv2

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

import json

In [15]:
current_dir = (os.getcwd())
current_dir

'd:\\DAFFA PINJEM BUAT BACKUP\\TA GASS\\FIX\\Yolo'

In [16]:
source_dir = './Defect-Coffee-Test/'
target_dir = './Coffee-Test-Result/'
predict_dir = './Predict/'

In [17]:
def delete_all_files(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
    except Exception as e:
      print('Failed to delete %s. Reason: %s' % (file_path, e))

In [18]:
# Initialize API for coffee bean detection
rf = Roboflow(api_key="IluK8shPQ2JbRq6f5FVM")

# Reference : https://universe.roboflow.com/coffe-dataset/coffee-beans-znwfe/model/2
project = rf.workspace().project("coffee-beans-znwfe")
model = project.version(2).model

# Reference : https://universe.roboflow.com/coffee-pqokc/coffee-beans-detection-test/model/5
# project = rf.workspace().project("coffee-beans-detection-test")
# model = project.version(5).model
confident = 20

def coffee_detection(filename):
  # infer on a local image
  result = model.predict(f"{source_dir}{filename}", confidence=confident).json()

  # visualize your prediction
  # model.predict(f"{source_dir}{filename}", confidence=confident, overlap=overlap).save(f"{target_dir}{filename}")
  return result

loading Roboflow workspace...
loading Roboflow project...


In [19]:
def get_pretrained_model(num_classes):
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)  # Ganti num_classes dengan jumlah kelas dalam dataset Anda
    return model

In [20]:
def load_model(checkpoint_path):
    model = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.eval()
    return model

In [21]:
# Coffee Bean Roast Level Detection
roast_label_list = ['Dark', 'Green', 'Light', 'Medium']
roastlevel_path = 'resnet50-coffee-roast-level.pth'
roastlevel_model = load_model(roastlevel_path)

# Coffee Bean Defect Detection
defect_label_list = ["Defect", "Normal"]
defect_path = 'resnet50-coffee-defect.pth'
defect_model = load_model(defect_path)

In [22]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize( [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [23]:
delete_all_files(predict_dir)
delete_all_files(target_dir)

In [24]:
file_images = os.listdir(source_dir)
img_list = []

for file_image in file_images:
    result = coffee_detection(file_image)

    # Open the image file
    for index, predict in enumerate(result["predictions"]):
        img = Image.open(os.path.join(source_dir, file_image))

        confidence = round(predict["confidence"] * 100, 2)

        # Set the crop region
        x, y, width, height = predict["x"], predict["y"], predict["width"], predict["height"]
        max_width = max(width, height)
        crop_region = (x-(int(max_width/2)), y-(int(max_width/2)), x+(int(max_width/2)), y+(int(max_width/2)))

        # Crop the image
        cropped_img = img.crop(crop_region)

        # Save cropped image
        cropped_filename = f"{file_image.split('.')[0]}-{index}.jpg"
        cropped_img.save(f"./Predict/{cropped_filename}")
        img_tuple = (cropped_filename, file_image, (x, y, width, height), confidence)
        img_list.append(img_tuple)

print(len(img_list))

110


In [25]:
img_detection = []
# loop through the images and make predictions
for img_predict in img_list:
    # load the image and apply the transformations
    image = Image.open(os.path.join(predict_dir, img_predict[0]))
    image = transform(image).unsqueeze(0)

    # make a prediction
    with torch.no_grad():
        # Coffee Bean Defect Detection
        defect_output = defect_model(image)
        predicted_defect = np.argmax(defect_output.numpy())
        defect_label = defect_label_list[predicted_defect.item()]

        # Coffee Bean Roast Level Detection
        # roast_output = roastlevel_model(image)
        # predicted_roast = np.argmax(roast_output.numpy())
        # roast_label = roast_label_list[predicted_roast.item()]

        # Get image source
        if not os.path.exists(os.path.join(target_dir, img_predict[1])):
            img_result = cv2.imread(os.path.join(source_dir, img_predict[1]))
        else:
            img_result = cv2.imread(os.path.join(target_dir, img_predict[1]))

        x, y, w, h = img_predict[2]

        # Draw bounding box on the image
        cv2.rectangle(img_result, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)), (0, 0, 255), 2)

        # Add label to the bounding box
        cv2.putText(img_result, f"{defect_label} {img_predict[3]}%", (x - int(w/2), y - int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        # cv2.putText(img_result, f"{roast_label} {img_predict[3]}%", (x - int(w/2), y - int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save the img_result with bounding box and label
        cv2.imwrite(os.path.join(target_dir, img_predict[1]), img_result)

        detect_tuple = (img_predict[0], img_predict[1], img_predict[2], img_predict[3],  defect_label)
        # detect_tuple = (img_predict[0], img_predict[1], img_predict[2], img_predict[3], roast_label)

        img_detection.append(detect_tuple)

In [26]:
img_detection = [list(item) for item in img_detection]
for item in img_detection:
    item[2] = list(item[2])

with open('result.json', 'w') as f:
    json.dump(img_detection, f)